# Heston 

In [2]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [57]:
from heston_model import *
import scipy.stats as stats

# test function

In [58]:
def d_plus(s,k,v):
    return np.log(s/k)/np.sqrt(v) + np.sqrt(v)/2

def d_minus(s,k,v):
    return np.log(s/k)/np.sqrt(v) - np.sqrt(v)/2
def Call(K,r = 0.05,q = 0, sigma = 0.3,S_0 = 100, T = 1.5):
    K_dc = K*np.exp(-r*T)
    S_f = S_0*np.exp(-q*T)
    return S_f*stats.norm.cdf(d_plus(S_f,K_dc,sigma**2*T))-K_dc*stats.norm.cdf(d_minus(S_f,K_dc,sigma**2*T))

In [74]:
params = {
    'T': 5.,
    'r': 0.05,
    'theta': 0.1,
    'kappa': 1,
    'sigma': 0.00001,
    'rho': -0.6,
    's0' : 1.0,
    'v0': 0.1
}

In [67]:
params_test = {
    'T': 5,
    'r': 0.05,
    'sigma': np.sqrt(0.1),
    'S_0': 1.,
    'K': 1
}

In [68]:
Call(**params_test)

0.3708495473708209

In [75]:
model = Heston(**params)
print(model.call(1))

0.37084896754038194
